In [1]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time

import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split



In [2]:
api_key = 'B6MOR128S6URKDZ0'

In [3]:
timeseries = TimeSeries(key=api_key, output_format = 'pandas')

data, meta_data = timeseries.get_intraday(symbol='MSFT', interval = '1min', outputsize = 'full')
print(data)

                      1. open   2. high    3. low  4. close  5. volume
date                                                                  
2019-12-09 16:00:00  151.4000  151.4000  151.2300  151.3600   345096.0
2019-12-09 15:59:00  151.3550  151.4000  151.3550  151.4000   104520.0
2019-12-09 15:58:00  151.3000  151.3500  151.2700  151.3500    83872.0
2019-12-09 15:57:00  151.3200  151.3300  151.2800  151.2905    58178.0
2019-12-09 15:56:00  151.3900  151.3900  151.2900  151.3100    93941.0
...                       ...       ...       ...       ...        ...
2019-12-03 09:35:00  147.4400  147.6182  147.3800  147.6182   123548.0
2019-12-03 09:34:00  147.1299  147.5703  147.0265  147.4353   131089.0
2019-12-03 09:33:00  147.0200  147.3300  147.0200  147.1100   197631.0
2019-12-03 09:32:00  147.3729  147.4860  146.7654  147.0129   407257.0
2019-12-03 09:31:00  147.1200  147.6300  147.0600  147.3800  1219138.0

[1941 rows x 5 columns]


In [4]:
#i = 1
# while i==1:
#     data, meta_data = timeseries.get_intraday(symbol='MSFT', interval = '1min', outputsize = 'full')
#     data.to_excel("output.xlsx")
#     time.sleep(60)
    

In [5]:
closing = data['4. close']

In [6]:
percentage_change = closing.pct_change()
print(percentage_change)

date
2019-12-09 16:00:00         NaN
2019-12-09 15:59:00    0.000264
2019-12-09 15:58:00   -0.000330
2019-12-09 15:57:00   -0.000393
2019-12-09 15:56:00    0.000129
                         ...   
2019-12-03 09:35:00   -0.000056
2019-12-03 09:34:00   -0.001239
2019-12-03 09:33:00   -0.002206
2019-12-03 09:32:00   -0.000660
2019-12-03 09:31:00    0.002497
Name: 4. close, Length: 1941, dtype: float64


In [7]:
last_one = percentage_change[-1]

In [8]:
if abs(last_one) > 0.0004:
    print(last_one)
    print("Change seen")
else: 
    print("No change")
    

0.00249705978182857
Change seen


In [9]:
f_out = 30 
data['Prediction'] = data[['4. close']].shift(-f_out)
print(data.tail())


                      1. open   2. high    3. low  4. close  5. volume  \
date                                                                     
2019-12-03 09:35:00  147.4400  147.6182  147.3800  147.6182   123548.0   
2019-12-03 09:34:00  147.1299  147.5703  147.0265  147.4353   131089.0   
2019-12-03 09:33:00  147.0200  147.3300  147.0200  147.1100   197631.0   
2019-12-03 09:32:00  147.3729  147.4860  146.7654  147.0129   407257.0   
2019-12-03 09:31:00  147.1200  147.6300  147.0600  147.3800  1219138.0   

                     Prediction  
date                             
2019-12-03 09:35:00         NaN  
2019-12-03 09:34:00         NaN  
2019-12-03 09:33:00         NaN  
2019-12-03 09:32:00         NaN  
2019-12-03 09:31:00         NaN  


In [10]:
X = np.array(data.drop(['Prediction'],1))
X = X[:-f_out]
print(X)

[[1.514000e+02 1.514000e+02 1.512300e+02 1.513600e+02 3.450960e+05]
 [1.513550e+02 1.514000e+02 1.513550e+02 1.514000e+02 1.045200e+05]
 [1.513000e+02 1.513500e+02 1.512700e+02 1.513500e+02 8.387200e+04]
 ...
 [1.478300e+02 1.479500e+02 1.478000e+02 1.478600e+02 5.313900e+04]
 [1.479800e+02 1.479900e+02 1.478200e+02 1.478400e+02 5.953700e+04]
 [1.478000e+02 1.479800e+02 1.477568e+02 1.479786e+02 9.022100e+04]]


In [11]:

y = np.array(data['Prediction'])
# To get all of the y values except the last n rows
y = y[:-f_out]
print(y)

[151.59   151.585  151.6042 ... 147.11   147.0129 147.38  ]


In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
svrrbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svrrbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [14]:
svm_confidence = svrrbf.score(x_test, y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.04406536777476833


In [15]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9683692491723679


In [17]:
x_forecast = np.array(data.drop(['Prediction'],1))[-f_out:]
print(x_forecast)

[[1.477300e+02 1.478300e+02 1.476300e+02 1.478100e+02 7.795800e+04]
 [1.475900e+02 1.476250e+02 1.475900e+02 1.476250e+02 7.397500e+04]
 [1.475900e+02 1.476500e+02 1.475800e+02 1.476500e+02 1.013730e+05]
 [1.473400e+02 1.476079e+02 1.473400e+02 1.476079e+02 6.463400e+04]
 [1.475100e+02 1.475100e+02 1.474100e+02 1.474100e+02 5.732200e+04]
 [1.474400e+02 1.475890e+02 1.474400e+02 1.475890e+02 5.053500e+04]
 [1.472600e+02 1.474300e+02 1.472600e+02 1.474300e+02 5.762700e+04]
 [1.473200e+02 1.473200e+02 1.473200e+02 1.473200e+02 6.643600e+04]
 [1.473153e+02 1.473600e+02 1.473153e+02 1.473600e+02 8.907300e+04]
 [1.472200e+02 1.473300e+02 1.472200e+02 1.473300e+02 9.343300e+04]
 [1.473200e+02 1.473200e+02 1.472600e+02 1.472600e+02 5.601600e+04]
 [1.473950e+02 1.473950e+02 1.472500e+02 1.472500e+02 1.141741e+06]
 [1.477055e+02 1.477055e+02 1.475600e+02 1.475600e+02 1.348550e+05]
 [1.474900e+02 1.477084e+02 1.474900e+02 1.477084e+02 6.210800e+04]
 [1.474900e+02 1.474900e+02 1.474500e+02 1.47450

In [18]:
# Print linear regression model predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[147.73974873 147.46754039 147.46254254 147.42330284 147.38778027
 147.44294294 147.27045736 147.1771733  147.18003105 147.1331534
 147.19560066 145.85089384 147.48574855 147.53587558 147.31653664
 147.37398845 147.3598506  147.18277966 147.10924061 147.2162498
 147.31738715 147.43675374 147.62969572 147.57934582 147.3878531
 147.43216571 147.24772874 146.75692446 147.05034743 145.71474411]


In [20]:
# Print support vector regressor model predictions for the next 'n' days
svm_prediction = svrrbf.predict(x_forecast)
print(svm_prediction)

[150.18091992 150.18149311 150.18149311 150.18149311 150.18149311
 150.18149311 150.18149311 150.18149311 150.18149311 150.18149311
 150.18149311 150.18149311 150.18149311 150.18105486 150.18149311
 150.18149311 150.18149311 150.18149311 150.18149311 150.18149311
 150.18149311 150.18149311 150.18149311 150.18149311 150.18149311
 150.18149311 150.18149311 150.18149311 150.18149311 150.18149311]
